<a href="https://colab.research.google.com/github/Hugo-magalhaes/Ifood_case/blob/main/case_ifood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Imports

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Window as W

import pandas as pd
import numpy as np

from func import extrai_info, full_analysis, text_to_json

import warnings

warnings.filterwarnings('ignore')

In [ ]:
# %%writefile func.py
from pyspark.sql import functions as F
from pyspark.sql import types as T

import requests
import gzip
import tarfile
import tempfile
import io
import os

from scipy.stats import ttest_ind

import numpy as np

def extrai_info(url, suffix_):
    '''
    Cria um arquivo temporário a partir da extensão do arquivo
    comprimido.

    Extensões comportadas:
    .json.gz
    .gz.csv
    .tar.gz
    '''

    resp = requests.get(url)
    resp.raise_for_status()
    data = resp.content

    if 'json' not in suffix_:
       data = gzip.decompress(data)

    if 'tar' in suffix_:
        tar_file = tarfile.open(fileobj=io.BytesIO(data))
        temp_dir = tempfile.mkdtemp()
        tar_file.extractall(temp_dir)
        tmp_path = os.path.join(temp_dir, os.listdir(temp_dir)[-1])

    else:
      with tempfile.NamedTemporaryFile(suffix=suffix_, delete=False) as tmp:
          tmp.write(data)
          tmp_path = tmp.name

          print('Arquivo temporário criado')

    return tmp_path


def text_to_json(col:str):
  '''
  Trata a informação da coluna items que está em formato string para array.
  '''
  item_schema = item_schema = T.ArrayType(
    T.StructType([
        T.StructField("name", T.StringType()),
        T.StructField("addition", T.StructType([
            T.StructField("value", T.IntegerType()),
            T.StructField("currency", T.StringType()),
        ])),
        T.StructField("discount", T.StructType([
            T.StructField("value", T.IntegerType()),
            T.StructField("currency", T.StringType()),
        ])),
        T.StructField("quantity", T.IntegerType()),
        T.StructField("sequence", T.IntegerType()),
        T.StructField("unitPrice", T.StructType([
            T.StructField("value", T.IntegerType()),
            T.StructField("currency", T.StringType()),
        ])),
        T.StructField("externalId", T.StringType()),
        T.StructField("totalValue", T.StructType([
            T.StructField("value", T.IntegerType()),
            T.StructField("currency", T.StringType()),
        ])),
        T.StructField("customerNote", T.StringType()),
        T.StructField("garnishItems", T.ArrayType(
            T.StructType([
                T.StructField("name", T.StringType()),
                T.StructField("addition", T.StructType([
                    T.StructField("value", T.IntegerType()),
                    T.StructField("currency", T.StringType()),
                ])),
                T.StructField("discount", T.StructType([
                    T.StructField("value", T.StringType()),
                    T.StructField("currency", T.StringType()),
                ])),
                T.StructField("quantity", T.IntegerType()),
                T.StructField("sequence", T.IntegerType()),
                T.StructField("unitPrice", T.StructType([
                    T.StructField("value", T.IntegerType()),
                    T.StructField("currency", T.StringType()),
                ])),
                T.StructField("categoryId", T.StringType()),
                T.StructField("externalId", T.StringType()),
                T.StructField("totalValue", T.StructType([
                    T.StructField("value", T.IntegerType()),
                    T.StructField("currency", T.StringType()),
                ])),
                T.StructField("categoryName", T.StringType()),
                T.StructField("integrationId", T.StringType()),
            ])
        )),
        T.StructField("integrationId", T.StringType()),
        T.StructField("totalAddition", T.StructType([
            T.StructField("value", T.IntegerType()),
            T.StructField("currency", T.StringType()),
        ])),
        T.StructField("totalDiscount", T.StructType([
            T.StructField("value", T.IntegerType()),
            T.StructField("currency", T.StringType()),
        ])),
    ])
)

  return F.from_json(F.col(col), item_schema)



def full_table(df_orders, df_ab_test, df_consumer, df_restaurant):

  return  (
      orders_df.select(
          F.col('customer_id'),
          F.col('merchant_id'),
          F.col('delivery_address_city'),
          F.col('delivery_address_district'),
          F.col('delivery_address_state'),
          F.col('delivery_address_zip_code'),
          F.date_format(F.col('order_created_at'), 'yyyy-MM-dd').alias('order_date'),
          text_to_json('items').alias('order_items'),
          F.col('order_scheduled'),
          F.col('origin_platform'),
          F.col('order_total_amount')

      ).join(
          consumer_df.select(
              F.col('customer_id'),
              F.col('active'),
              F.col('customer_phone_area')

          ),
          on='customer_id', how='left'
      ).join(
          ab_test_df,
          on='customer_id', how='left'
      ).join(
          restaurant_df.select(
              F.col('id').alias('merchant_id'),
              F.col('enabled'),
              F.col('price_range'),
              F.col('average_ticket'),
              F.col('takeout_time'),
              F.col('delivery_time'),
              F.col('minimum_order_value'),
              F.col('merchant_city'),
              F.col('merchant_state')

          ),
          on='merchant_id', how='left'
      )
  )

def cohens_d(a, b):
    '''
    Calcula a diferença das médias pelo desvio padrão
    das duas distribuições.
    Mede impacto da ação
    '''
    cohend = (a.mean() - b.mean()) / np.sqrt((a.var() + b.var()) / 2)
    return cohend

def stat_analysis(df_pd, col_aval:str):
    '''
    Calcula Cohend para impacto e t-test para significância.
    '''
    target = df_pd[df_pd.is_target=="target"][col_aval]
    control = df_pd[df_pd.is_target=="control"][col_aval]

    d = cohens_d(target, control)

    t_stat, p_val = ttest_ind(target, control, equal_var=False)

    lenght = '-'*(25-len(col_aval)//2)

    print(lenght, col_aval, lenght)
    print(f"T-stat: {t_stat:.3f}")
    print(f"p-value: {p_val:.3f}")
    print(f"Cohen's d: {d:.3f}")
    print()

    return t_stat, p_val, d

def calculate_metrics(full_df):
  date_metrics = (full_df.filter(F.col('is_target').isNotNull()

                  ).groupBy('order_date', 'is_target').agg(
                      F.count('*').alias('totl_order_day')
                    , F.sum('order_total_amount').alias('GMV')
                    , F.countDistinct('customer_id').alias('totl_order_clie')

                  ).withColumn('AOV'
                               , F.round(F.col('GMV')/F.col('totl_order_day'), 4)

                  )
                  )

  camp_metrics = (date_metrics.groupBy('is_target').agg(
                            F.round(F.mean('GMV'), 4).alias('GMV_day')
                            , F.round(F.mean('AOV'), 4).alias('AOV_day')
                            , F.sum('totl_order_day').alias('totl_order')
                            , F.sum('totl_order_clie').alias('totl_clients')
                            , F.round(F.sum('GMV'), 4).alias('totl_paid')
                            , F.round(F.mean('totl_order_day'), 4).alias('mean_order_day')
                        )
                  )
  return camp_metrics, date_metrics

def calculate_dif_audience(camp_metrics, margin:float=0.15, total_days:int=60):

  camp_metrics_pd = camp_metrics_pd.toPandas()

  gmv_control = camp_metrics_pd[camp_metrics_pd['is_target'] == 'control']['totl_paid'].iloc[0]
  gmv_target = camp_metrics_pd[camp_metrics_pd['is_target'] == 'target']['totl_paid'].iloc[0]

  mean_order_control = camp_metrics_pd[camp_metrics_pd['is_target'] == 'control']['mean_order_day'].iloc[0]
  mean_order_target = camp_metrics_pd[camp_metrics_pd['is_target'] == 'target']['mean_order_day'].iloc[0]

  # Calculate incremental GMV
  gmv_incremental = (gmv_target - gmv_control)
  # Calculate incremental mean orders per day
  mean_order_incremental = mean_order_target - mean_order_control

  # Calculate Margin
  Margin = gmv_incremental * margin

  print(f"GMV Incremental : R$ {gmv_incremental:,.2f}")
  print(f"GMV Incremental por dia: R$ {gmv_incremental/total_days:,.2f}")
  print(f"Média de Pedidos por Dia Incremental: {mean_order_incremental:,.2f}")
  print(f"Margin (20% de margem): R$ {Margin:,.2f}")
  print(f"Margin (20% de margem) por dia: R$ {Margin/total_days:,.2f}")

  return gmv_incremental, gmv_incremental/total_days, mean_order_incremental, Margin, Margin/total_days

def segment_audience(full_df):
  segmented_user_metrics = (
    full_df.filter(F.col('is_target').isNotNull()

                  ).groupBy('customer_id', 'is_target').agg(
                      F.count('*').alias('totl_order_user')
                    , F.sum('order_total_amount').alias('GMV')
                    # , F.avg('price_range').cast('int').alias('avg_price_range')
                    # , F.avg('average_ticket').cast('int').alias('avg_average_ticket')
                    # , F.avg('takeout_time').cast('int').alias('avg_takeout_time')
                    # , F.avg('delivery_time').cast('int').alias('avg_delivery_time')
                    # , F.avg('minimum_order_value').cast('int').alias('avg_minimum_order_value')

                  ).withColumn('AOV'
                               , F.round(F.col('GMV')/F.col('totl_order_user'), 4)

                  ).withColumn('paid_client', F.round(F.col('GMV')/F.col('totl_order_user'), 4)
                  ).withColumn("order_band",
                                F.when(F.col("totl_order_user") <= 1, "1. Ped. eventual")
                                .when(F.col("totl_order_user") <= 2, "2. Ped. mensal")
                                .when(F.col("totl_order_user") <= 5, "3. Ped. quinzenal")
                                .otherwise("4. Ped. direto")
                            )
                )
  return segmented_user_metrics


def calculate_dif_segment_audience(segmented_user_metrics):
  segmented_metrics_agg = (segmented_user_metrics
                        .groupBy('is_target', 'order_band')
                        .agg(F.sum('GMV').alias('GMV_comp')
                            , F.count('*').alias('clients_comp')
                            , F.sum('totl_order_user').alias('totl_order_comp')
                        ).orderBy(F.col('order_band'), F.col('is_target')
                        ).withColumn('GMV_user', F.round(F.col('GMV_comp')/F.col('clients_comp'), 4))
                        .withColumn('Pedidos_comp', F.round(F.col('totl_order_comp')/F.col('clients_comp'), 4))
                        .withColumn('AOV_user', F.round(F.col('GMV_comp')/F.col('totl_order_comp'), 4))
)

  segmented_metrics_pd = segmented_metrics_agg.toPandas()


  grupos = {}
  print("Análise de Diferença entre Target e Control por Faixa de Pedidos:")
  for band in segmented_metrics_pd['order_band'].unique():
      print(f"\n----- {band} -----")
      control_data = segmented_metrics_pd[(segmented_metrics_pd['order_band'] == band) & (segmented_metrics_pd['is_target'] == 'control')]
      target_data = segmented_metrics_pd[(segmented_metrics_pd['order_band'] == band) & (segmented_metrics_pd['is_target'] == 'target')]

      if not control_data.empty and not target_data.empty:
          control_gmv_user = control_data['GMV_user'].iloc[0]
          target_gmv_user = target_data['GMV_user'].iloc[0]
          diff_gmv_user = target_gmv_user - control_gmv_user
          pct_diff_gmv_user = (diff_gmv_user / control_gmv_user) * 100 if control_gmv_user != 0 else np.nan

          control_pedidos_comp = control_data['Pedidos_comp'].iloc[0]
          target_pedidos_comp = target_data['Pedidos_comp'].iloc[0]
          diff_pedidos_comp = target_pedidos_comp - control_pedidos_comp
          pct_diff_pedidos_comp = (diff_pedidos_comp / control_pedidos_comp) * 100 if control_pedidos_comp != 0 else np.nan

          control_aov_user = control_data['AOV_user'].iloc[0]
          target_aov_user = target_data['AOV_user'].iloc[0]
          diff_aov_user = target_aov_user - control_aov_user
          pct_diff_aov_user = (diff_aov_user / control_aov_user) * 100 if control_aov_user != 0 else np.nan


          grupos[band] = [control_aov_user, target_aov_user, diff_aov_user, pct_diff_aov_user]

          print(f"GMV por Usuário (Diferença Absoluta): {diff_gmv_user:,.2f} | (Diferença %): {pct_diff_gmv_user:,.2f}%")
          print(f"Pedidos por Usuário (Diferença Absoluta): {diff_pedidos_comp:,.2f} | (Diferença %): {pct_diff_pedidos_comp:,.2f}%")
          print(f"AOV por Usuário (Diferença Absoluta): {diff_aov_user:,.2f} | (Diferença %): {pct_diff_aov_user:,.2f}%")
      else:
          print("Dados insuficientes para esta banda.")

  return segmented_metrics_pd, grupos


In [ ]:
spark = (SparkSession.builder.master("local[*]").appName('CaseIfood')
       # '*' significa: use todos os núcleos/CPUs disponíveis no ambiente
    .config("spark.executor.memory", "6g")
    .config("spark.driver.memory", "6g")
    .config("spark.executor.cores", "4")
    .config("spark.driver.cores", "4")

    # 3. Otimizações de desempenho geral
    .config("spark.sql.shuffle.partitions", "200")
    .config("spark.default.parallelism", "100")
    .getOrCreate())

<h1> Loading Tables

In [ ]:
consumer_url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz"
restaurant_url = 'https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz'
ab_test_url = 'https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz'
orders_url = 'https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz'

restaurant_path = extrai_info(restaurant_url, ".gz.csv")
consumer_path = extrai_info(consumer_url, ".gz.csv")
ab_test_path = extrai_info(ab_test_url, ".tar.gz")
orders_path = extrai_info(orders_url, ".json.gz")

Não é tipo JSON
Temporário criado
Não é tipo JSON
Temporário criado
Não é tipo JSON
Compressão tar
Temporário criado


In [ ]:
restaurant_df = spark.read.csv(restaurant_path, header=True, inferSchema=True)
consumer_df = spark.read.csv(consumer_path, header=True, inferSchema=True)
ab_test_df = spark.read.csv(ab_test_path, header=True, inferSchema=True)
orders_df = spark.read.json(orders_path)

<h2> Validating strucutre

In [ ]:
print('Restaurantes')
restaurant_df.printSchema()
print('Clientes')
consumer_df.printSchema()
print('Teste A/B')
ab_test_df.printSchema()
print('Pedidos')
orders_df.printSchema()

Restaurantes
root
 |-- id: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- enabled: boolean (nullable = true)
 |-- price_range: integer (nullable = true)
 |-- average_ticket: double (nullable = true)
 |-- takeout_time: integer (nullable = true)
 |-- delivery_time: integer (nullable = true)
 |-- minimum_order_value: double (nullable = true)
 |-- merchant_zip_code: integer (nullable = true)
 |-- merchant_city: string (nullable = true)
 |-- merchant_state: string (nullable = true)
 |-- merchant_country: string (nullable = true)

Clientes
root
 |-- customer_id: string (nullable = true)
 |-- language: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- active: boolean (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_phone_area: integer (nullable = true)
 |-- customer_phone_number: integer (nullable = true)

Teste A/B
root
 |-- customer_id: string (nullable = true)
 |-- is_target: string (nullable = true)

Pedidos


In [ ]:
totl_restaurant = restaurant_df.count()

totl_customers = consumer_df.count()

totl_ab_test = ab_test_df.count()


print(f'{totl_restaurant = :,}')
print(f'{totl_customers = :,}')
print(f'{totl_ab_test = :,}')

totl_restaurant = 7,292
totl_customers = 806,156
totl_ab_test = 806,467


<h2> Building Full Table

In [ ]:
def full_table(df_orders, df_ab_test, df_consumer, df_restaurant):

  return  (
      orders_df.select(
          F.col('customer_id'),
          F.col('merchant_id'),
          F.col('delivery_address_city'),
          F.col('delivery_address_district'),
          F.col('delivery_address_state'),
          F.col('delivery_address_zip_code'),
          F.date_format(F.col('order_created_at'), 'yyyy-MM-dd').alias('order_date'),
          text_to_json('items').alias('order_items'),
          F.col('order_scheduled'),
          F.col('origin_platform'),
          F.col('order_total_amount')

      ).join(
          consumer_df.select(
              F.col('customer_id'),
              F.col('active'),
              F.col('customer_phone_area')

          ),
          on='customer_id', how='left'
      ).join(
          ab_test_df,
          on='customer_id', how='left'
      ).join(
          restaurant_df.select(
              F.col('id').alias('merchant_id'),
              F.col('enabled'),
              F.col('price_range'),
              F.col('average_ticket'),
              F.col('takeout_time'),
              F.col('delivery_time'),
              F.col('minimum_order_value'),
              F.col('merchant_city'),
              F.col('merchant_state')

          ),
          on='merchant_id', how='left'
      )
  )

def calculate_metrics(full_df):
  date_metrics = (full_df.filter(F.col('is_target').isNotNull()

                  ).groupBy('order_date', 'is_target').agg(
                      F.count('*').alias('totl_order_day')
                    , F.sum('order_total_amount').alias('GMV')
                    , F.countDistinct('customer_id').alias('totl_order_clie')

                  ).withColumn('AOV'
                               , F.round(F.col('GMV')/F.col('totl_order_day'), 4)

                  )
                  )

  camp_metrics = (date_metrics.groupBy('is_target').agg(
                            F.round(F.mean('GMV'), 4).alias('GMV_day')
                            , F.round(F.mean('AOV'), 4).alias('AOV_day')
                            , F.sum('totl_order_day').alias('totl_order')
                            , F.sum('totl_order_clie').alias('totl_clients')
                            , F.round(F.sum('GMV'), 4).alias('totl_paid')
                            , F.round(F.mean('totl_order_day'), 4).alias('mean_order_day')
                        )
                  )
  return camp_metrics, date_metrics

def calculate_dif_audience(camp_metrics, margin:float=0.15, total_days:int=60):

  camp_metrics_pd = camp_metrics_pd.toPandas()

  gmv_control = camp_metrics_pd[camp_metrics_pd['is_target'] == 'control']['totl_paid'].iloc[0]
  gmv_target = camp_metrics_pd[camp_metrics_pd['is_target'] == 'target']['totl_paid'].iloc[0]

  mean_order_control = camp_metrics_pd[camp_metrics_pd['is_target'] == 'control']['mean_order_day'].iloc[0]
  mean_order_target = camp_metrics_pd[camp_metrics_pd['is_target'] == 'target']['mean_order_day'].iloc[0]

  # Calculate incremental GMV
  gmv_incremental = (gmv_target - gmv_control)
  # Calculate incremental mean orders per day
  mean_order_incremental = mean_order_target - mean_order_control

  # Calculate Margin
  Margin = gmv_incremental * margin

  print(f"GMV Incremental : R$ {gmv_incremental:,.2f}")
  print(f"GMV Incremental por dia: R$ {gmv_incremental/total_days:,.2f}")
  print(f"Média de Pedidos por Dia Incremental: {mean_order_incremental:,.2f}")
  print(f"Margin (20% de margem): R$ {Margin:,.2f}")
  print(f"Margin (20% de margem) por dia: R$ {Margin/total_days:,.2f}")

  return gmv_incremental, gmv_incremental/total_days, mean_order_incremental, Margin, Margin/total_days

def segment_audience(full_df):
  segmented_user_metrics = (
    full_df.filter(F.col('is_target').isNotNull()

                  ).groupBy('customer_id', 'is_target').agg(
                      F.count('*').alias('totl_order_user')
                    , F.sum('order_total_amount').alias('GMV')
                    # , F.avg('price_range').cast('int').alias('avg_price_range')
                    # , F.avg('average_ticket').cast('int').alias('avg_average_ticket')
                    # , F.avg('takeout_time').cast('int').alias('avg_takeout_time')
                    # , F.avg('delivery_time').cast('int').alias('avg_delivery_time')
                    # , F.avg('minimum_order_value').cast('int').alias('avg_minimum_order_value')

                  ).withColumn('AOV'
                               , F.round(F.col('GMV')/F.col('totl_order_user'), 4)

                  ).withColumn('paid_client', F.round(F.col('GMV')/F.col('totl_order_user'), 4)
                  ).withColumn("order_band",
                                F.when(F.col("totl_order_user") <= 1, "1. Ped. eventual")
                                .when(F.col("totl_order_user") <= 2, "2. Ped. mensal")
                                .when(F.col("totl_order_user") <= 5, "3. Ped. quinzenal")
                                .otherwise("4. Ped. direto")
                            )
                )
  return segmented_user_metrics


def calculate_dif_segment_audience(segmented_user_metrics):
  segmented_metrics_agg = (segmented_user_metrics
                        .groupBy('is_target', 'order_band')
                        .agg(F.sum('GMV').alias('GMV_comp')
                            , F.count('*').alias('clients_comp')
                            , F.sum('totl_order_user').alias('totl_order_comp')
                        ).orderBy(F.col('order_band'), F.col('is_target')
                        ).withColumn('GMV_user', F.round(F.col('GMV_comp')/F.col('clients_comp'), 4))
                        .withColumn('Pedidos_comp', F.round(F.col('totl_order_comp')/F.col('clients_comp'), 4))
                        .withColumn('AOV_user', F.round(F.col('GMV_comp')/F.col('totl_order_comp'), 4))
)

  segmented_metrics_pd = segmented_metrics_agg.toPandas()


  grupos = {}
  print("Análise de Diferença entre Target e Control por Faixa de Pedidos:")
  for band in segmented_metrics_pd['order_band'].unique():
      print(f"\n----- {band} -----")
      control_data = segmented_metrics_pd[(segmented_metrics_pd['order_band'] == band) & (segmented_metrics_pd['is_target'] == 'control')]
      target_data = segmented_metrics_pd[(segmented_metrics_pd['order_band'] == band) & (segmented_metrics_pd['is_target'] == 'target')]

      if not control_data.empty and not target_data.empty:
          control_gmv_user = control_data['GMV_user'].iloc[0]
          target_gmv_user = target_data['GMV_user'].iloc[0]
          diff_gmv_user = target_gmv_user - control_gmv_user
          pct_diff_gmv_user = (diff_gmv_user / control_gmv_user) * 100 if control_gmv_user != 0 else np.nan

          control_pedidos_comp = control_data['Pedidos_comp'].iloc[0]
          target_pedidos_comp = target_data['Pedidos_comp'].iloc[0]
          diff_pedidos_comp = target_pedidos_comp - control_pedidos_comp
          pct_diff_pedidos_comp = (diff_pedidos_comp / control_pedidos_comp) * 100 if control_pedidos_comp != 0 else np.nan

          control_aov_user = control_data['AOV_user'].iloc[0]
          target_aov_user = target_data['AOV_user'].iloc[0]
          diff_aov_user = target_aov_user - control_aov_user
          pct_diff_aov_user = (diff_aov_user / control_aov_user) * 100 if control_aov_user != 0 else np.nan


          grupos[band] = [control_aov_user, target_aov_user, diff_aov_user, pct_diff_aov_user]

          print(f"GMV por Usuário (Diferença Absoluta): {diff_gmv_user:,.2f} | (Diferença %): {pct_diff_gmv_user:,.2f}%")
          print(f"Pedidos por Usuário (Diferença Absoluta): {diff_pedidos_comp:,.2f} | (Diferença %): {pct_diff_pedidos_comp:,.2f}%")
          print(f"AOV por Usuário (Diferença Absoluta): {diff_aov_user:,.2f} | (Diferença %): {pct_diff_aov_user:,.2f}%")
      else:
          print("Dados insuficientes para esta banda.")

  return segmented_metrics_pd, grupos

<h1> Premissas <h1>

1. Dado que não é dito quando foi lançado o cupom especial, então será considerado o período de dez/18 a jan/19 como avaliação de viabialidade

2. Como o teste A/B não tem data nem marcação por pedido, apenas por cliente, entende-se que, o período todo em todo pedido o cliente era possível de utilizar o cupom.

3. Dado que o cupom é especial, entende-se que qualquer cliente gostaria de economizar e utilizou em todo pedido. Enquanto, os clientes de teste não obtiveram o desconto.

4. 8.505 clientes não têm rastreabilidade da campanha, logo serão desconsiderados.

5. Margem teórica de 20%.

6. A campanha deve levar em conta a rastreabilidade de custo para ela. Uma vez que, seja pelo preço unitário, valor total ou deconto, não é possível medir a perda esperada pela campanha.

7. Para manter uma maior constância de pedidos, além de, maior constância de fluxo de caixa é válido ofertar esse cupom especial para clientes dentro da assinatura para que o retenha dentro da assinatura também.

8. Para medir a eficácia dessa proposta, podemos medir apenas com grupo que paga assinatura, tendo um grupo controle e um de teste a diferença de churn na assinatura pelo mesmo período.É possível também medir pela média de pedidos x AOV e medir impacto com e sem cupom é favorável para o resultado.

9. Como exemplo, atualmente temos 13 MM de assinantes em uma basde 55 MM de clientes ativos. Cerca de 23,6% do clientes são assinantes. Dado que plataformas em geral tem churn de 5% a 7%. Esperamos uma taxa de 6%.

10. O clube atual usa 5 cupons de R\$ 10,00 por R\$ 4,95 por mês. Em geral, vemos um cupom especial a cada 15 dias sem utilizar a plataforma. Os assinantes não têm acesso a esse cupom, e passariam a ter e seria superior a R\$ 10,00, usando como premissa dois cupons de r$\ 15,00 por mês.


In [ ]:
df_with_discounts = (
    full_df.withColumn(
    "total_discount_amount",
    F.expr("""
        round(aggregate(
            order_items,
            0D,
            (acc, item) -> acc
                + COALESCE(CAST(item.discount.value AS DOUBLE) / 100, 0D)
                + COALESCE(CAST(item.totalDiscount.value AS DOUBLE) /100 , 0D)
                + COALESCE(
                    aggregate(
                            item.garnishItems,
                            0D,
                            (g_acc, g_item) -> g_acc + COALESCE(CAST(g_item.discount.value AS DOUBLE) / 100, 0D)
                        ),
                    0D
                )
        ), 2)
    """)
).withColumn(
    "total_value_amount_items",
    F.expr("""
        round(aggregate(
            order_items,
            0D,
            (acc, item) -> acc
                + COALESCE(CAST(item.unitPrice.value AS DOUBLE) / 100 * CAST(item.quantity AS INT) , 0D)
        ), 2)
    """)
).withColumn(
    "total_paid_amount",
    F.round(F.col('order_total_amount') - F.col('total_discount_amount'),2) # Use order_total_amount for total paid
)
)

df_with_discounts.select('customer_id', 'order_total_amount', 'total_discount_amount', 'total_value_amount_items', 'total_paid_amount').show(10)

+--------------------+------------------+---------------------+------------------------+-----------------+
|         customer_id|order_total_amount|total_discount_amount|total_value_amount_items|total_paid_amount|
+--------------------+------------------+---------------------+------------------------+-----------------+
|004dc916e193d924b...|              46.1|                  0.0|                    46.1|             46.1|
|004dc916e193d924b...|              46.1|                  0.0|                    46.1|             46.1|
|00515897b190757b5...|             51.98|                  0.0|                   51.98|            51.98|
|0051f30fea7dbc96c...|              26.5|                  0.0|                    26.5|             26.5|
|0051f30fea7dbc96c...|              38.0|                  0.0|                    38.0|             38.0|
|0051f30fea7dbc96c...|              10.0|                  0.0|                     0.0|             10.0|
|0051f30fea7dbc96c...|              2

In [ ]:
orders_df.limit(1).show()

+-----------+--------------------+-------------+---------------------+------------------------+-------------------------+----------------------------+-------------------------+--------------------------+----------------------+-------------------------+--------------------+--------------------+-----------------+------------------+-----------------+--------------------+--------------------+---------------+--------------------+------------------+---------------+
|        cpf|         customer_id|customer_name|delivery_address_city|delivery_address_country|delivery_address_district|delivery_address_external_id|delivery_address_latitude|delivery_address_longitude|delivery_address_state|delivery_address_zip_code|               items|         merchant_id|merchant_latitude|merchant_longitude|merchant_timezone|    order_created_at|            order_id|order_scheduled|order_scheduled_date|order_total_amount|origin_platform|
+-----------+--------------------+-------------+---------------------+--

In [ ]:
full_df.select(F.col('order_items'), F.col('customer_id'), F.col('is_target')).limit(1).collect()

[Row(order_items=[Row(name='Parmegiana de Filé de Frango (2 pessoas)', addition=Row(value='0', currency='BRL'), discount=Row(value='0', currency='BRL'), quantity='1.0', sequence='1', unitPrice=Row(value='2800', currency='BRL'), externalId='0bcd6764fd5e466d9c04b18ac0eb69e6', totalValue=Row(value='2800', currency='BRL'), customerNote=None, garnishItems=[Row(name='COM Arroz branco', addition=Row(value='0', currency='BRL'), discount=Row(value='0', currency='BRL'), quantity='1.0', sequence='2', unitPrice=Row(value='0', currency='BRL'), categoryId='13HDH', externalId='384bd2b4eb7d454d8e0274e7d590ab4f', totalValue=Row(value='0', currency='BRL'), categoryName='PERSONALIZAR', integrationId=None)], integrationId='PMFR', totalAddition=Row(value='0', currency='BRL'), totalDiscount=Row(value='0', currency='BRL')), Row(name='Lasanha Frango (2 pessoas)', addition=Row(value='0', currency='BRL'), discount=Row(value='0', currency='BRL'), quantity='1.0', sequence='3', unitPrice=Row(value='1800', currency

In [ ]:
full_df.filter(F.col('is_target').isin('target')).select(F.col('order_items'), F.col('customer_id'), F.col('is_target')).limit(1).collect()

[Row(order_items=[Row(name='Na compra de uma pizza familia ganhe um Guaraná 2 litros ou um Suco de caixa 1 litro', addition=Row(value='0', currency='BRL'), discount=Row(value='0', currency='BRL'), quantity='1.0', sequence='1', unitPrice=Row(value='0', currency='BRL'), externalId='3fdfc925fc364f27a5c960cfc881df52', totalValue=Row(value='0', currency='BRL'), customerNote='Sem azeitona', garnishItems=[Row(name='21 - QUATRO QUEIJOS', addition=Row(value='0', currency='BRL'), discount=Row(value='0', currency='BRL'), quantity='1.0', sequence='2', unitPrice=Row(value='5500', currency='BRL'), categoryId='1JL26', externalId='7b0adaa810434aa7998e5064f2383e00', totalValue=Row(value='5500', currency='BRL'), categoryName='Escolha sua opção', integrationId=None), Row(name='Guaraná 2 litros', addition=Row(value='0', currency='BRL'), discount=Row(value='0', currency='BRL'), quantity='1.0', sequence='3', unitPrice=Row(value='0', currency='BRL'), categoryId='1JL9N', externalId='f8fa7d8c26854f22af36d863fc

In [ ]:
orders_df.filter(F.col('customer_id').isin('0044d74253c1439957b0687e29600f8837411092f7cb47cf74efd3659441752e')).show()

+-----------+--------------------+-------------+---------------------+------------------------+-------------------------+----------------------------+-------------------------+--------------------------+----------------------+-------------------------+--------------------+--------------------+-----------------+------------------+-----------------+--------------------+--------------------+---------------+--------------------+------------------+---------------+
|        cpf|         customer_id|customer_name|delivery_address_city|delivery_address_country|delivery_address_district|delivery_address_external_id|delivery_address_latitude|delivery_address_longitude|delivery_address_state|delivery_address_zip_code|               items|         merchant_id|merchant_latitude|merchant_longitude|merchant_timezone|    order_created_at|            order_id|order_scheduled|order_scheduled_date|order_total_amount|origin_platform|
+-----------+--------------------+-------------+---------------------+--

In [ ]:
restaurant_df.groupBy('merchant_state', 'merchant_country').count().show()

+--------------+----------------+-----+
|merchant_state|merchant_country|count|
+--------------+----------------+-----+
|            PA|              BR|   50|
|            SC|              BR|  183|
|            PB|              BR|   85|
|            RJ|              BR| 1170|
|            PI|              BR|    3|
|            MT|              BR|   15|
|            BA|              BR|  190|
|            MS|              BR|   41|
|            MA|              BR|   20|
|            DF|              BR|  372|
|            SE|              BR|   30|
|            MG|              BR|  556|
|            PR|              BR|  372|
|            PE|              BR|  179|
|            GO|              BR|  152|
|            AL|              BR|   29|
|            AM|              BR|   56|
|            RN|              BR|   90|
|            SP|              BR| 3127|
|            AC|              BR|    4|
+--------------+----------------+-----+
only showing top 20 rows



In [ ]:
consumer_df.groupBy('language').count.show()

+--------+------+
|language| count|
+--------+------+
|   pt-br|806154|
|   es-ar|     2|
+--------+------+



In [ ]:
full_df.show()

+--------------------+--------------------+---------------------+-------------------------+----------------------+-------------------------+----------+--------------------+---------------+---------------+------------------+------+-------------------+---------+-------+-----------+--------------+------------+-------------+-------------------+-------------------+--------------+
|         merchant_id|         customer_id|delivery_address_city|delivery_address_district|delivery_address_state|delivery_address_zip_code|order_date|         order_items|order_scheduled|origin_platform|order_total_amount|active|customer_phone_area|is_target|enabled|price_range|average_ticket|takeout_time|delivery_time|minimum_order_value|      merchant_city|merchant_state|
+--------------------+--------------------+---------------------+-------------------------+----------------------+-------------------------+----------+--------------------+---------------+---------------+------------------+------+--------------

In [ ]:
(full_df
 .groupBy('is_target').agg(F.count('*').alias('totl_pedidos')
                          , F.sum('order_total_amount').alias('totl_paid')
                          ).withColumn(
                              'AOV', F.round(F.col('totl_paid')/F.col('totl_pedidos'), 4)
                          )
).show()

+---------+------------+--------------------+-------+
|is_target|totl_pedidos|           totl_paid|    AOV|
+---------+------------+--------------------+-------+
|  control|     1525576| 7.307187288000315E7|47.8979|
|   target|     2136745|1.0200756984000635E8|47.7397|
|     NULL|        8505|   753325.1300000189|88.5744|
+---------+------------+--------------------+-------+



In [ ]:
user_metrics = (
    full_df.filter(F.col('is_target').isNotNull()

                  ).groupBy('customer_id', 'is_target').agg(
                      F.count('*').alias('totl_order_user')
                    , F.sum('order_total_amount').alias('GMV')
                    # , F.avg('price_range').cast('int').alias('avg_price_range')
                    # , F.avg('average_ticket').cast('int').alias('avg_average_ticket')
                    # , F.avg('takeout_time').cast('int').alias('avg_takeout_time')
                    # , F.avg('delivery_time').cast('int').alias('avg_delivery_time')
                    # , F.avg('minimum_order_value').cast('int').alias('avg_minimum_order_value')

                  ).withColumn('AOV'
                               , F.round(F.col('GMV')/F.col('totl_order_user'), 4)

                  ).withColumn('paid_client', F.round(F.col('GMV')/F.col('totl_order_user'), 4)
                  )
)

In [ ]:
camp_metrics_validation = (user_metrics.groupBy('is_target').agg(
                      F.round(F.mean('GMV'), 4).alias('GMV_user')
                      , F.round(F.mean('AOV'), 4).alias('AOV_user')
                      , F.sum('totl_order_user').alias('totl_order')
                      , F.count('*').alias('totl_clients')
                      , F.round(F.sum('GMV'), 4).alias('totl_paid')
                      , F.round(F.mean('totl_order_user'), 4).alias('mean_order_user')
                  )
)

camp_metrics_validation.show()

+---------+--------+--------+----------+------------+--------------+---------------+----------+
|is_target|GMV_user|AOV_user|totl_order|totl_clients|     totl_paid|mean_order_user|mean_spend|
+---------+--------+--------+----------+------------+--------------+---------------+----------+
|  control|202.6723| 48.0114|   1525576|      360542| 7.307187288E7|         4.2313|  202.6723|
|   target|228.7555| 47.8336|   2136745|      445924|1.0200756984E8|         4.7917|  228.7555|
+---------+--------+--------+----------+------------+--------------+---------------+----------+



In [ ]:
camp_metrics_pd = camp_metrics_validation.toPandas()

In [ ]:
camp_metrics_pd

,is_target,GMV_day,AOV_day,totl_order,totl_clients,totl_paid,mean_order_day
0,control,1217864.548,47.3685,1525576,1404725,7.307187e+07,25426.2667
1,target,1700126.164,47.1843,2136745,1955936,1.020076e+08,35612.4167


In [ ]:

gmv_control = camp_metrics_pd[camp_metrics_pd['is_target'] == 'control']['totl_paid'].iloc[0]
gmv_target = camp_metrics_pd[camp_metrics_pd['is_target'] == 'target']['totl_paid'].iloc[0]

mean_order_control = camp_metrics_pd[camp_metrics_pd['is_target'] == 'control']['mean_order_day'].iloc[0]
mean_order_target = camp_metrics_pd[camp_metrics_pd['is_target'] == 'target']['mean_order_day'].iloc[0]

# Calculate incremental GMV
gmv_incremental = (gmv_target - gmv_control)
# Calculate incremental mean orders per day
mean_order_incremental = mean_order_target - mean_order_control

# Calculate Margin
Margin = gmv_incremental * 0.15

print(f"GMV Incremental : R$ {gmv_incremental:,.2f}")
print(f"GMV Incremental por dia: R$ {gmv_incremental/60:,.2f}")
print(f"Média de Pedidos por Dia Incremental: {mean_order_incremental:,.2f}")
print(f"Margin (20% de margem): R$ {Margin:,.2f}")
print(f"Margin (20% de margem) por dia: R$ {Margin/60:,.2f}")

GMV Incremental : R$ 28,935,696.96
GMV Incremental por dia: R$ 482,261.62
Média de Pedidos por Dia Incremental: 10,186.15
Margin (20% de margem): R$ 4,340,354.54
Margin (20% de margem) por dia: R$ 72,339.24


In [ ]:
df_pd = camp_metrics_validation.toPandas()

In [ ]:
full_analysis(df_pd, 'GMV')
full_analysis(df_pd, 'AOV')
full_analysis(df_pd, 'totl_order_clie')

------------------------ GMV ------------------------
T-stat: 4.339
p-value: 0.000
Cohen's d: 0.792

------------------------ AOV ------------------------
T-stat: -0.309
p-value: 0.758
Cohen's d: -0.056

------------------ totl_order_clie ------------------
T-stat: 4.732
p-value: 0.000
Cohen's d: 0.864



In [ ]:
df_state_orders = (
    full_df.groupBy('delivery_address_state', 'is_target')
    .agg(F.count('*').alias('total_orders'))
    .toPandas()
)

<h1> Segmentation

In [ ]:
user_metrics.groupBy('totl_order_user').count().orderBy(F.desc('count')).show()

+---------------+------+
|totl_order_user| count|
+---------------+------+
|              2|239462|
|              1|182587|
|              4| 81538|
|              3| 70961|
|              6| 40151|
|              5| 38955|
|              7| 23734|
|              8| 23240|
|              9| 15488|
|             10| 14405|
|             11| 10390|
|             12|  9713|
|             13|  7185|
|             14|  6694|
|             15|  5189|
|             16|  4739|
|             17|  3756|
|             18|  3473|
|             19|  2896|
|             20|  2510|
+---------------+------+
only showing top 20 rows



In [ ]:
segmented_user_metrics = user_metrics.withColumn(
    "AOV_band",
    F.when(F.col("AOV") <= 29.2, "1. AOV baixo")
    .when(F.col("AOV") <= 40.825, "2. AOV médio")
    .when(F.col("AOV") <= 56.73, "3. AOV alto")
    .otherwise("4. AOV altíssimo")
).withColumn(
    "order_band",
    F.when(F.col("totl_order_user") <= 1, "1. Ped. eventual")
    .when(F.col("totl_order_user") <= 2, "2. Ped. mensal")
    .when(F.col("totl_order_user") <= 5, "3. Ped. quinzenal")
    .otherwise("4. Ped. direto")
)

In [ ]:
(segmented_user_metrics
                        .groupBy('is_target', 'order_band')
                        .agg(F.sum('GMV').alias('GMV_comp')
                            , F.count('*').alias('clients_comp')
                            , F.sum('totl_order_user').alias('totl_order_comp')
                        ).orderBy(F.col('order_band'), F.col('is_target')
                        ).withColumn('GMV_user', F.round(F.col('GMV_comp')/F.col('clients_comp'), 4)
                        ).withColumn('Pedidos_comp', F.round(F.col('totl_order_comp')/F.col('clients_comp'), 4)
                        ).withColumn('AOV_user', F.round(F.col('GMV_comp')/F.col('totl_order_comp'), 4))

 ).show()

+---------+----------------+--------------------+------------+---------------+--------+------------+--------+
|is_target|      order_band|            GMV_comp|clients_comp|totl_order_comp|GMV_user|Pedidos_comp|AOV_user|
+---------+----------------+--------------------+------------+---------------+--------+------------+--------+
|  control|1. Ped. eventual|   4385038.330000012|       91201|          91201|  48.081|         1.0|  48.081|
|   target|1. Ped. eventual|   4391110.350000014|       91386|          91386| 48.0501|         1.0| 48.0501|
|  control|  2. Ped. mensal| 1.096954094000004E7|      115368|         230736|  95.083|         2.0| 47.5415|
|   target|  2. Ped. mensal|1.1794468370000033E7|      124094|         248188| 95.0446|         2.0| 47.5223|
|  control|  3. Ped. mensal|1.4299245490000004E7|       76352|         292670|187.2806|      3.8332| 48.8579|
|   target|  3. Ped. mensal|2.1184167059999995E7|      115102|         441140|184.0469|      3.8326| 48.0214|
|  control

In [ ]:
segmented_metrics_agg = (segmented_user_metrics
                        .groupBy('is_target', 'order_band')
                        .agg(F.sum('GMV').alias('GMV_comp')
                            , F.count('*').alias('clients_comp')
                            , F.sum('totl_order_user').alias('totl_order_comp')
                        ).orderBy(F.col('order_band'), F.col('is_target')
                        ).withColumn('GMV_user', F.round(F.col('GMV_comp')/F.col('clients_comp'), 4))
                        .withColumn('Pedidos_comp', F.round(F.col('totl_order_comp')/F.col('clients_comp'), 4))
                        .withColumn('AOV_user', F.round(F.col('GMV_comp')/F.col('totl_order_comp'), 4))
)

segmented_metrics_pd = segmented_metrics_agg.toPandas()

print("Análise de Diferença entre Target e Control por Faixa de Pedidos:")
for band in segmented_metrics_pd['order_band'].unique():
    print(f"\n----- {band} -----")
    control_data = segmented_metrics_pd[(segmented_metrics_pd['order_band'] == band) & (segmented_metrics_pd['is_target'] == 'control')]
    target_data = segmented_metrics_pd[(segmented_metrics_pd['order_band'] == band) & (segmented_metrics_pd['is_target'] == 'target')]

    if not control_data.empty and not target_data.empty:
        control_gmv_user = control_data['GMV_user'].iloc[0]
        target_gmv_user = target_data['GMV_user'].iloc[0]
        diff_gmv_user = target_gmv_user - control_gmv_user
        pct_diff_gmv_user = (diff_gmv_user / control_gmv_user) * 100 if control_gmv_user != 0 else np.nan

        control_pedidos_comp = control_data['Pedidos_comp'].iloc[0]
        target_pedidos_comp = target_data['Pedidos_comp'].iloc[0]
        diff_pedidos_comp = target_pedidos_comp - control_pedidos_comp
        pct_diff_pedidos_comp = (diff_pedidos_comp / control_pedidos_comp) * 100 if control_pedidos_comp != 0 else np.nan

        control_aov_user = control_data['AOV_user'].iloc[0]
        target_aov_user = target_data['AOV_user'].iloc[0]
        diff_aov_user = target_aov_user - control_aov_user
        pct_diff_aov_user = (diff_aov_user / control_aov_user) * 100 if control_aov_user != 0 else np.nan

        print(f"GMV por Usuário (Diferença Absoluta): {diff_gmv_user:,.2f} | (Diferença %): {pct_diff_gmv_user:,.2f}%")
        print(f"Pedidos por Usuário (Diferença Absoluta): {diff_pedidos_comp:,.2f} | (Diferença %): {pct_diff_pedidos_comp:,.2f}%")
        print(f"AOV por Usuário (Diferença Absoluta): {diff_aov_user:,.2f} | (Diferença %): {pct_diff_aov_user:,.2f}%")
    else:
        print("Dados insuficientes para esta banda.")

Análise de Diferença entre Target e Control por Faixa de Pedidos:

----- 1. Ped. eventual -----
GMV por Usuário (Diferença Absoluta): -0.03 | (Diferença %): -0.06%
Pedidos por Usuário (Diferença Absoluta): 0.00 | (Diferença %): 0.00%
AOV por Usuário (Diferença Absoluta): -0.03 | (Diferença %): -0.06%

----- 2. Ped. mensal -----
GMV por Usuário (Diferença Absoluta): -0.04 | (Diferença %): -0.04%
Pedidos por Usuário (Diferença Absoluta): 0.00 | (Diferença %): 0.00%
AOV por Usuário (Diferença Absoluta): -0.02 | (Diferença %): -0.04%

----- 3. Ped. mensal -----
GMV por Usuário (Diferença Absoluta): -3.23 | (Diferença %): -1.73%
Pedidos por Usuário (Diferença Absoluta): -0.00 | (Diferença %): -0.02%
AOV por Usuário (Diferença Absoluta): -0.84 | (Diferença %): -1.71%

----- 4. Ped. direto -----
GMV por Usuário (Diferença Absoluta): 1.04 | (Diferença %): 0.19%
Pedidos por Usuário (Diferença Absoluta): 0.02 | (Diferença %): 0.17%
AOV por Usuário (Diferença Absoluta): 0.01 | (Diferença %): 0.01

In [ ]:
import plotly.express as px

# Aggregate the segmented_metrics_pd by order_band and is_target for GMV_comp
pie_data = segmented_metrics_pd.groupby(['order_band'])['clients_comp'].sum().reset_index()

# Create the pie chart
fig_pie = px.pie(
    pie_data,
    values='clients_comp',
    names='order_band',
    title='Proportion of GMV by Order Band and A/B Test Group',
    hole=0.3, # Creates a donut chart
    # color='is_target', # Segment by is_target group
    # facet_col='is_target', # Create separate pie charts for each is_target group
    # color_discrete_map={'control': 'salmon', 'target': 'firebrick'},
    labels={'clients_comp': 'Total Clients', 'order_band': 'Order Frequency Band'},
    height=500
)

fig_pie.show()

In [ ]:
import plotly.express as px

# Filter out 'is_target' with null values as per premise 4
df_state_orders_filtered = df_state_orders[df_state_orders['is_target'].notnull()]

# Create the bar chart for total orders by state, segmented by is_target
fig = px.bar(
    df_state_orders_filtered,
    x='delivery_address_state',
    y='total_orders',
    color='is_target', # Segment by 'is_target'
    title='Total Orders by State Segmented by A/B Test Group',
    labels={
        'delivery_address_state': 'State',
        'total_orders': 'Total Orders',
        'is_target': 'A/B Test Group'
    },
    height=600,
    color_discrete_map={'control': 'salmon', 'target': 'firebrick'},
    barmode='group' # Adjusted to display bars side-by-side
)

fig.update_layout(xaxis_title_text='State', yaxis_title_text='Total Orders')
fig.show()

In [ ]:
df_time_orders = (
    full_df.groupBy('order_date', 'is_target')
    .agg(F.count('*').alias('total_orders'))
    .orderBy('order_date')
    .toPandas()
)

# Filter out 'is_target' with null values as per premise 4
df_time_orders_filtered = df_time_orders[df_time_orders['is_target'].notnull()]

# Create the line chart for total orders over time, segmented by is_target
fig = px.line(
    df_time_orders_filtered,
    x='order_date',
    y='total_orders',
    color='is_target',
    title='Total Orders Over Time Segmented by A/B Test Group',
    labels={
        'order_date': 'Date',
        'total_orders': 'Total Orders',
        'is_target': 'A/B Test Group'
    },
    height=600,
    color_discrete_map={'control': 'salmon', 'target': 'firebrick'}
)

fig.update_layout(xaxis_title_text='Date', yaxis_title_text='Total Orders')
fig.show()